In [10]:
import pygame
import random
import sys
from collections import deque

# Constants
CELL_SIZE = 24
SIDEBAR_WIDTH = 200

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
ORANGE = (255, 165, 0)

pygame.init()
font_large = pygame.font.SysFont(None, 60)
font_medium = pygame.font.SysFont(None, 40)
font_small = pygame.font.SysFont(None, 30)

# Directions
DIRECTIONS = [(0, -1), (1, 0), (0, 1), (-1, 0)]

class Game:
    def __init__(self):
        self.mode = "easy"
        self.set_dimensions()
        self.screen_width = self.COLS * CELL_SIZE + SIDEBAR_WIDTH
        self.screen_height = self.ROWS * CELL_SIZE
        self.grid = [[1 for _ in range(self.COLS)] for _ in range(self.ROWS)]
        self.pacman_pos = (1, 1)
        self.ghosts = []
        self.points = set()
        self.big_stars = set()
        self.game_over = False
        self.total_score = 0
        self.big_collected = 0
        self.ghost_timers = []

    def set_dimensions(self):
        if self.mode == "easy":
            self.ROWS, self.COLS = 11, 11
        elif self.mode == "moderate":
            self.ROWS, self.COLS = 15, 15
        else:
            self.ROWS, self.COLS = 21, 21

    def initialize_game(self):
        self.set_dimensions()
        self.screen_width = self.COLS * CELL_SIZE + SIDEBAR_WIDTH
        self.screen_height = self.ROWS * CELL_SIZE
        self.grid = [[1 for _ in range(self.COLS)] for _ in range(self.ROWS)]
        random.seed(None)  # Ensure randomness
        self.generate_maze()
        self.add_loops()
        self.place_collectibles()
        self.pacman_pos = (1, 1)
        self.ghosts = []
        self.ghost_timers = []
        self.game_over = False
        self.total_score = 0
        self.big_collected = 0

        if self.mode in ["moderate", "ai"]:
            self.ghosts.append((self.COLS - 2, self.ROWS - 2))
            self.ghost_timers.append(0)
        elif self.mode == "advanced":
            self.ghosts = [(self.COLS - 2, self.ROWS - 2), (1, self.ROWS - 2), (self.COLS - 2, 1)]
            self.ghost_timers = [0, 3, 6]

    def generate_maze(self):
        def in_bounds(x, y):
            return 0 <= x < self.COLS and 0 <= y < self.ROWS

        def carve_maze(x, y):
            self.grid[y][x] = 0
            dirs = random.sample(DIRECTIONS, len(DIRECTIONS))
            for dx, dy in dirs:
                nx, ny = x + dx*2, y + dy*2
                if in_bounds(nx, ny) and self.grid[ny][nx] == 1:
                    self.grid[y + dy][x + dx] = 0
                    carve_maze(nx, ny)

        self.grid = [[1 for _ in range(self.COLS)] for _ in range(self.ROWS)]
        start_x, start_y = random.randrange(1, self.COLS, 2), random.randrange(1, self.ROWS, 2)
        carve_maze(start_x, start_y)
        self.ensure_connectivity()

    def add_loops(self):
        attempts = 0
        added = 0
        max_loops = 10 if self.mode == "easy" else 20 if self.mode == "moderate" else 30
        while added < max_loops and attempts < 500:
            x = random.randrange(1, self.COLS - 1, 2)
            y = random.randrange(1, self.ROWS - 1, 2)
            dirs = random.sample(DIRECTIONS, 2)
            for dx, dy in dirs:
                nx, ny = x + dx, y + dy
                if 0 < nx < self.COLS - 1 and 0 < ny < self.ROWS - 1:
                    if self.grid[ny][nx] == 1:
                        self.grid[ny][nx] = 0
                        added += 1
            attempts += 1

    def ensure_connectivity(self):
        visited = [[False]*self.COLS for _ in range(self.ROWS)]
        queue = deque([(1, 1)])
        visited[1][1] = True

        while queue:
            x, y = queue.popleft()
            for dx, dy in DIRECTIONS:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.COLS and 0 <= ny < self.ROWS and not visited[ny][nx] and self.grid[ny][nx] == 0:
                    visited[ny][nx] = True
                    queue.append((nx, ny))

        for y in range(self.ROWS):
            for x in range(self.COLS):
                if self.grid[y][x] == 0 and not visited[y][x]:
                    self.grid[y][x] = 1

    def place_collectibles(self):
        self.points = set()
        self.big_stars = set()
        for y in range(self.ROWS):
            for x in range(self.COLS):
                if self.grid[y][x] == 0 and (x, y) != self.pacman_pos:
                    self.points.add((x, y))
        for _ in range(5):
            if self.points:
                star = random.choice(list(self.points))
                self.big_stars.add(star)
                self.points.remove(star)

    def move_pacman(self, dx, dy):
        x, y = self.pacman_pos
        nx, ny = x + dx, y + dy

        if nx < 0: nx = self.COLS - 1
        elif nx >= self.COLS: nx = 0
        if ny < 0 or ny >= self.ROWS: return

        if self.grid[ny][nx] == 0:
            self.pacman_pos = (nx, ny)
            if (nx, ny) in self.big_stars:
                self.total_score += 50
                self.big_collected += 1
                self.big_stars.discard((nx, ny))
            elif (nx, ny) in self.points:
                self.total_score += 10
                self.points.discard((nx, ny))

    def move_ghosts(self):
        new_ghosts = []
        for i, ghost in enumerate(self.ghosts):
            self.ghost_timers[i] += 1
            if self.ghost_timers[i] % (25 + i*10) == 0:
                new_pos = self.bfs_toward_pacman(ghost)
                new_ghosts.append(new_pos)
            else:
                new_ghosts.append(ghost)
        self.ghosts = new_ghosts

    def bfs_toward_pacman(self, start):
        queue = deque([(start, [])])
        visited = set()
        visited.add(start)

        while queue:
            (x, y), path = queue.popleft()
            if (x, y) == self.pacman_pos:
                return path[0] if path else (x, y)
            for dx, dy in DIRECTIONS:
                nx, ny = x + dx, y + dy
                if (0 <= nx < self.COLS and 0 <= ny < self.ROWS and
                    self.grid[ny][nx] == 0 and (nx, ny) not in visited):
                    visited.add((nx, ny))
                    queue.append(((nx, ny), path + [(nx, ny)]))
        return start

    def check_collisions(self):
        for ghost in self.ghosts:
            if ghost == self.pacman_pos:
                self.game_over = True
                
    def draw(self, screen):
        screen.fill(BLUE)
        for y in range(self.ROWS):
            for x in range(self.COLS):
                rect = pygame.Rect(x*CELL_SIZE, y*CELL_SIZE, CELL_SIZE, CELL_SIZE)
                if self.grid[y][x] == 1:
                    pygame.draw.rect(screen, BLACK, rect)
                elif (x, y) in self.points:
                    pygame.draw.circle(screen, WHITE, rect.center, 4)
                elif (x, y) in self.big_stars:
                    pygame.draw.circle(screen, GREEN, rect.center, 7)

        ghost_colors = [RED, CYAN, MAGENTA]
        for i, ghost in enumerate(self.ghosts):
            color = ghost_colors[i % len(ghost_colors)]
            gx, gy = ghost
            pygame.draw.circle(screen, color, (gx*CELL_SIZE + CELL_SIZE//2, gy*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 4)

        px, py = self.pacman_pos
        pygame.draw.circle(screen, YELLOW, (px*CELL_SIZE + CELL_SIZE//2, py*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 4)

        sidebar_x = self.COLS * CELL_SIZE + 20
        pygame.draw.rect(screen, BLACK, (self.COLS * CELL_SIZE, 0, SIDEBAR_WIDTH, self.screen_height))
        score_text = font_small.render(f"Score: {self.total_score}", True, WHITE)
        mode_text = font_small.render(f"Mode: {self.mode.capitalize()}", True, WHITE)
        big_text = font_small.render(f"Big Stars: {self.big_collected}", True, WHITE)
        screen.blit(score_text, (sidebar_x, 40))
        screen.blit(mode_text, (sidebar_x, 80))
        screen.blit(big_text, (sidebar_x, 120))

        if self.game_over:
            over_text = font_large.render("GAME OVER", True, RED)
            retry_text = font_medium.render("Press R to Retry or Q to Quit", True, WHITE)
            screen.blit(over_text, over_text.get_rect(center=(self.COLS * CELL_SIZE // 2, self.screen_height // 2 - 40)))
            screen.blit(retry_text, retry_text.get_rect(center=(self.COLS * CELL_SIZE // 2, self.screen_height // 2 + 20)))
        elif not self.points and not self.big_stars:
            win_text = font_large.render("YOU WIN!", True, GREEN)
            score_text = font_medium.render(f"Score: {self.total_score}", True, WHITE)
            screen.blit(win_text, win_text.get_rect(center=(self.COLS * CELL_SIZE // 2, self.screen_height // 2 - 120)))
            screen.blit(score_text, score_text.get_rect(center=(self.COLS * CELL_SIZE // 2, self.screen_height // 2 - 60)))

def find_path_bfs(maze, start, goal):
    queue = deque([(start, [])])
    visited = set()
    while queue:
        (x, y), path = queue.popleft()
        if (x, y) == goal:
            return path
        if (x, y) in visited:
            continue
        visited.add((x, y))
        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy
            if 0 <= ny < len(maze) and 0 <= nx < len(maze[0]) and maze[ny][nx] == 0:
                queue.append(((nx, ny), path + [(dx, dy)]))
    return []

def main():
    screen = pygame.display.set_mode((800, 600))
    pygame.display.set_caption("Pacman Maze")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont(None, 50)

    modes = [
        ("easy", "EASY (0 Ghosts)"),
        ("moderate", "MODERATE (1 Ghost)"),
        ("advanced", "ADVANCED (3 Ghosts)"),
        ("ai", "AI MODE (1 Ghost + AI Pacman)")
    ]
    selected_mode = 0
    selecting = True

    while selecting:
        screen.fill(BLACK)
        title = font_large.render("Select Mode", True, WHITE)
        screen.blit(title, title.get_rect(center=(400, 100)))

        mode_colors = [GREEN, ORANGE, RED, BLUE]

        for i, (_, mode_display) in enumerate(modes):
            base_color = mode_colors[i]
            color = base_color if i == selected_mode else (base_color[0]//2, base_color[1]//2, base_color[2]//2)
            mode_text = font_medium.render(mode_display, True, color)
            screen.blit(mode_text, mode_text.get_rect(center=(400, 200 + i * 60)))
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_DOWN:
                    selected_mode = (selected_mode + 1) % len(modes)
                elif event.key == pygame.K_UP:
                    selected_mode = (selected_mode - 1) % len(modes)
                elif event.key == pygame.K_RETURN:
                    selecting = False

    game = Game()
    game.mode = modes[selected_mode][0]
    game.initialize_game()
    screen = pygame.display.set_mode((game.screen_width, game.screen_height))
    ai_move_counter = 0

    while True:
        screen.fill(BLACK)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    pygame.quit()
                    sys.exit()
                elif event.key == pygame.K_r and game.game_over:
                    game.initialize_game()
                elif not game.game_over and game.mode != "ai":
                    if event.key == pygame.K_UP:
                        game.move_pacman(0, -1)
                    elif event.key == pygame.K_DOWN:
                        game.move_pacman(0, 1)
                    elif event.key == pygame.K_LEFT:
                        game.move_pacman(-1, 0)
                    elif event.key == pygame.K_RIGHT:
                        game.move_pacman(1, 0)

        if not game.game_over and (game.points or game.big_stars):
            if game.mode == "ai":
                ai_move_counter += 1
                if ai_move_counter % 3 == 0:  # Adjust this number to control AI speed (higher = slower)
                    targets = list(game.points) + list(game.big_stars)
                    if targets:
                        targets.sort(key=lambda t: abs(t[0] - game.pacman_pos[0]) + abs(t[1] - game.pacman_pos[1]))
                        path = find_path_bfs(game.grid, game.pacman_pos, targets[0])
                        if path:
                            dx, dy = path[0]
                            game.move_pacman(dx, dy)
            game.move_ghosts()
            game.check_collisions()

        game.draw(screen)
        pygame.display.flip()
        clock.tick(10)

if __name__ == "__main__":
    main()




SystemExit: 